In [1]:
import random,timeit
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sklearn import svm
from sklearn import model_selection as ms

In [2]:
def random_search(n,dim):
    gens=[[0 if g != j else 1 for g in range(n)] for j in range(dim)]
    return gens



In [3]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.KFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= svm.LinearSVC().fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [4]:
def bees_optimization(bee,binary,i):
    binary=list(binary)
    j=random.randint(0,len(binary)-1)
    k=random.randint(0,len(binary)-1)
    while k==i:
        k=random.randint(0,len(binary)-1)
    fit=binary[j]+random.uniform(-1,1)*(binary[j]-binary[k])
    for x in range(bee):
        y=random.randint(0,len(binary)-1)
        while y==i:
            y=random.randint(0,len(binary)-1)
        r=random.uniform(0,1)
        if r<=fit:
            binary[y]=1
        
    return binary

In [5]:
def BABCO(train_d,train_l,n=10,max_iter=25,employee_percent=0.5,max_limit=5):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    employed_bees = int(round(n*employee_percent))
    onlooker_bees = n - employed_bees       

    dim=len(train_d[0])
    global_best=float("-inf")
    global_position=tuple([0]*dim)
    gens_dict = {}
    limit=[0]*dim
    gens=random_search(dim,dim)
    for gen in gens:
        if tuple(gen) in gens_dict:
            score = gens_dict[tuple(gen)]
        else:
            score=evaluate(train_d,train_l,gen)
            gens_dict[tuple(gen)]=score
        if score > global_best:
            global_best=score
            global_position=dc(gen)
            
            
    for it in range(max_iter):
        for i in range(employed_bees):
            for i,x in enumerate(gens):
                gen=bees_optimization(employed_bees,x,i)
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gen)
                    gens_dict[tuple(gen)]=score

                if score > gens_dict[tuple(gens[i])]:
                    limit[i]=0
                    gens[i]= gen
                else:
                    limit[i]+=1

                if score > global_best:
                    global_best=score
                    global_position=dc(gen)

                if limit[i]>=max_limit:
                    gens[i]=[0 if g != i else 1 for g in range(dim)]
    
        for i in range(onlooker_bees):
            for i,x in enumerate(gens):
                gen=bees_optimization(employed_bees,x,i)
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gen)
                    gens_dict[tuple(gen)]=score

                if score > gens_dict[tuple(gens[i])]:
                    limit[i]=0
                    gens[i]= gen
                else:
                    limit[i]+=1

                if score > global_best:
                    global_best=score
                    global_position=dc(gen)

                if limit[i]>=max_limit:
                    gens[i]=[0 if g != i else 1 for g in range(dim)]

                
    return global_position,global_position.count(1)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
import numpy as np

def test_score(gen, tr_x, tr_y, te_x, te_y):
    mask = np.array(gen) == 1
    al_data = np.array(tr_x[:, mask])
    al_test_data = np.array(te_x[:, mask])

    accuracies, precisions, recalls, f1s = [], [], [], []
    for _ in range(5):
        model = SVC(kernel='rbf', C=1.0, gamma='scale') 
        model.fit(al_data, tr_y)
        preds = model.predict(al_test_data)

        accuracies.append(accuracy_score(te_y, preds))
        precisions.append(precision_score(te_y, preds, zero_division=0))
        recalls.append(recall_score(te_y, preds, zero_division=0))
        f1s.append(f1_score(te_y, preds, zero_division=0))

    return (
        np.mean(accuracies),
        np.mean(precisions),
        np.mean(recalls),
        np.mean(f1s)
    )

In [13]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

with open("/Users/ximenacanton/Documents/TEC/5 SEM/bio inspirado/reto/Bio-inspired-algorithms-for-diagnosis-of-breast-cancer/wdbc.data.csv") as f:
    lines = f.read().splitlines()

# id (columna 0), diagnosis (columna 1), resto son features numéricas
x = np.array([[float(d) for d in line.split(',')[2:]] for line in lines])  # desde la 3ra columna
y = np.array([line.split(',')[1] for line in lines])  # diagnosis ('M' o 'B')

lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)

train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)


print("Shape X:", x.shape)

Shape X: (569, 30)


In [14]:
# Evaluación inicial con todas las características
k = [1 for r in range(len(x[0]))]
acc, prec, rec, f1 = test_score(k, train_d, train_l, test_d, test_l)
print(f"All features → Acc:{acc:.4f}  Prec:{prec:.4f}  Rec:{rec:.4f}  F1:{f1:.4f}")

# Inicialización de métricas
fattr = 0
ftest_acc = 0.0
ftest_prec = 0.0
ftest_rec = 0.0
ftest_f1 = 0.0

flist = [0 for _ in range(len(x[0]))]
final_list = [0 for _ in range(len(x[0]))]

start = timeit.default_timer()

# Repeticiones
for i in range(20):
    g, l = BABCO(train_d, train_l, n=10, max_iter=25, employee_percent=0.5, max_limit=5)
    fattr += l

    acc, prec, rec, f1 = test_score(g, train_d, train_l, test_d, test_l)
    ftest_acc += acc
    ftest_prec += prec
    ftest_rec += rec
    ftest_f1 += f1

    for j in range(len(flist)):
        if g[j] == 1:
            flist[j] += 1

    print(f"{i+1:02d}  {''.join(map(str,g))}  {l}  Acc:{acc:.4f}  F1:{f1:.4f}")

# Promedios finales
fattr /= 20
ftest_acc /= 20
ftest_prec /= 20
ftest_rec /= 20
ftest_f1 /= 20

end = timeit.default_timer()
time = end - start

# Selección final de características
final_count = int(round(fattr))
final_count = max(0, min(final_count, len(flist)))
final = np.argsort(flist)[::-1][:final_count]
for i in range(len(final)):
    final_list[final[i]] = 1

print(f"\nFinal: {''.join(map(str,final_list))}")
print(f"Selected features: {fattr:.2f}")
print(f"Average metrics → Acc:{ftest_acc:.4f}  Prec:{ftest_prec:.4f}  Rec:{ftest_rec:.4f}  F1:{ftest_f1:.4f}")
print(f"Execution time: {time:.4f} s")

All features → Acc:0.9091  Prec:0.9556  Rec:0.7963  F1:0.8687
01  111101000011011110000100000010  13  Acc:0.8741  F1:0.8125
02  111111111111010011111010110110  23  Acc:0.8741  F1:0.8125
03  111110010010100000101000000010  11  Acc:0.8741  F1:0.8125
04  111100001100001000011000101011  13  Acc:0.8741  F1:0.8125
05  111101111100101111111010011011  22  Acc:0.8671  F1:0.8000
06  110010010000000010011010010111  12  Acc:0.9161  F1:0.8824
07  111111010010000001011000100010  13  Acc:0.8741  F1:0.8125
08  000011100001100111011110110110  16  Acc:0.9371  F1:0.9109
09  000000000001010000100101010010  7  Acc:0.9021  F1:0.8600
10  010111100011111100110111111110  21  Acc:0.9091  F1:0.8687
11  001101100100000000001011010011  11  Acc:0.9091  F1:0.8687
12  010011101110101100011001100111  17  Acc:0.9021  F1:0.8600
13  010000011101010000100101011010  12  Acc:0.9021  F1:0.8600
14  011010100000001100001000000111  10  Acc:0.8951  F1:0.8454
15  111100110000001011111010001000  14  Acc:0.8671  F1:0.8000
16  10010